# 内容
## PyTorchの基本
- ネットワークアーキテクチャの定義
- データセットとデータローダー
- トレーニング
- 推論

## データ増強
- データ増強

## マサチューセッツ州の建物のデータセット
このチュートリアルでは、建物の検出にデータセットを使用します。

In [ ]:
import numpy as np
from PIL import Image
%matplotlib inline
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

fpath_image = '../dataset/building_vmnih/test/image/22828930_15.tiff'
fpath_label = '../dataset/building_vmnih/test/label/22828930_15.tif'

image = np.array(Image.open(fpath_image))
label = np.array(Image.open(fpath_label))

fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(121)
ax.imshow(image, interpolation='none')
ax.set_xticks([])
ax.set_yticks([])
fig.show()
ax.set_title('Source image')

ax = fig.add_subplot(122)
ax.imshow(label, interpolation='none')
ax.set_xticks([])
ax.set_yticks([])
ax.set_title('Ground truth')

fig.show()


データセットでは、137組の画像と上記のようなラベルがトレーニング用に提供されています。データセットでモデルをトレーニングする前に、pytorchの基本について説明します。

##  PyTorchの基本
PyTorchは、Facebookが開発した有名なディープラーニングフレームワークです。ここでは、PyTorchの基本を紹介します。

次に、pytorchモジュールをインポートします。 <br>
`torch.nn`には、畳み込み、プーリング、バッチ正規化など、さまざまな種類の操作のクラスが含まれます。<br>
`torch.nn.functional`には、アクティベーション関数などの基本的な（およびノンパラメトリックな）操作のための関数が含まれています。

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

### PyTorchのテンソル
テンソルはさまざまな方法で作成できます。 テンソルは、numpy配列のように使用できます。
```
torch.FloatTensor([1,2,3])
torch.from_numpy(ndarray)
torch.Tensor([1,2,3]).float()
```

### 基本操作
ネットワークを実装する前に、畳み込みやプーリングなどの基本操作を定義する方法を簡単に確認します。 <br><br>

**畳み込み層**<br>
畳み込み層はクラス（ `nn.Conv2d`）として実装されます。 最初にレイヤーのインスタンスを作成し、それを関数のように使用します。
```
conv1 = torch.nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
output = conv1(input)
```

**プール層**
```
pool1 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
output = pool1(input)

pool2 = torch.nn.AvgPool2d(kernel_size=2, stride=2)
output = pool2(input)
```

**完全に接続されたレイヤー**
```
fc1 = torch.nn.Linear(in_features=128, out_features=128)
output = fc1(input)
```

**アクティベーション機能**<br>
アクティベーションは関数として実装されます。
```
output = torch.nn.functional.relu(input)
```
クラスバージョンも使用できます。
```
act1 = torch.nn.ReLU()
output = act1(input)
```



### ネットワークアーキテクチャの定義
上記の基本操作を使用して、入力として$64\times64$パッチを持つVGGのようなモデルを実装し、サイズ$16\times16$の中心領域の推定値を出力します。

In [ ]:

class VGGS(nn.Module):
    def __init__(self):
        super(VGGS, self).__init__()
        self.conv1_1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.conv1_2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1)
        self.pool1   = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2_1 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv2_2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        self.pool2   = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3_1 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.conv3_2 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1)
        self.pool3   = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv4_1 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.conv4_2 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        self.pool4   = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc6 = nn.Linear(in_features=4096, out_features=1024)
        self.fc7 = nn.Linear(in_features=1024, out_features=1024)
        self.fc8 = nn.Linear(in_features=1024, out_features=256)
        

    def forward(self, x):
        x = F.relu(self.conv1_1(x))
        x = F.relu(self.conv1_2(x))
        x = self.pool1(x)
        x = F.relu(self.conv2_1(x))
        x = F.relu(self.conv2_2(x))
        x = self.pool2(x)
        x = F.relu(self.conv3_1(x))
        x = F.relu(self.conv3_2(x))
        x = self.pool3(x)
        x = F.relu(self.conv4_1(x))
        x = F.relu(self.conv4_2(x))
        x = self.pool4(x)

        batch_size = x.size(0)
        x = x.view(batch_size, -1)
        
        x = F.dropout(self.fc6(x))
        x = F.dropout(self.fc7(x))
        x = self.fc8(x)
        x = x.view(batch_size, 16, 16)
        return x
    
        

モデルを作成し、アーキテクチャを確認する

In [ ]:
model = VGGS()
print(model)

ランダムテンソルを処理して、アーキテクチャが正しいかどうかを確認します。

In [ ]:
inputs = np.random.randn(100,3,64,64).astype(np.float32)
inputs = torch.from_numpy(inputs)
output = model(inputs)
print(output.size())

### 損失関数
ここでは、トレーニングのクロスエントロピー損失関数を定義します。 `log()`でのオーバーフローを避けるために、小さな値`eps`が必要です。

In [ ]:
import torch.nn.functional as F

class Criterion(nn.Module):
    def __init__(self):
        super(Criterion, self).__init__()
        self.eps = 1.0e-7
        
    def forward(self, inputs, targets):
        prob = F.sigmoid(inputs)
        loss = -targets.float()*(prob + self.eps).log() - (1-targets.float())*(1 - prob + self.eps).log()
        loss = loss.mean()
        return loss


## データセットとデータローダー
以下の図は、トレーニング用のミニバッチがPyTorchでどのように作成されるかを示しています。 <br>
**DataLoader**: DataLoaderはDatasetからデータを取得し、収集したデータをミニバッチにパックします。 <br>
**Dataset**: データローダから要求されたデータセットのロードと前処理データ。 場合によっては、独自のデータ用にカスタマイズされたデータセットを実装する必要があります。 <br>

<img src="../fig/pth_dataset.png">

### 独自のデータセットを定義する

In [ ]:
import numpy as np
from torch.utils.data import Dataset

class Buildings(Dataset):
    def __init__(self, fpath_image_npy, fpath_label_npy):
        self.images = np.load(fpath_image_npy)
        self.labels = np.load(fpath_label_npy)
        
    def __getitem__(self, index):
        image = self.images[index]
        label = self.labels[index]
        
        image = image.transpose([2,0,1])    # (H,W,B) to (B,H,W)
        
        image_tensor = torch.from_numpy(image).float()
        label_tensor = torch.from_numpy(label).long()
        
        return image_tensor, label_tensor
    
    def __len__(self):
        return len(self.images)

建物データセットを読み込む

In [ ]:
dataset = Buildings('../dataset/building_vmnih/train/patches/sat.npy', '../dataset/building_vmnih/train/patches/map.npy')
image_tensor, label_tensor = dataset[0]
print(image_tensor.size())
print(label_tensor.size())
print(len(dataset))

次に、DataLoaderを設定し、機能するかどうかを確認します。 <br><br>
DataLoaderのinitの引数: <br>
batch_size: batch_sizeを指定します<br>
shuffle: Trueの場合、データはDatasetからランダムにサンプリングされ、Falseの場合、データは順次サンプリングされます。<br>
num_workers: マルチスレッドワーカーの数。 データを並行してロードできるため、データのロードが高速化されます。

In [ ]:
from torch.utils.data import DataLoader

loader = DataLoader(dataset, batch_size=10, shuffle=True, num_workers=0)
for image, label in loader:
    print(image.size(), torch.mean(image).item())
    print(label.size(), label[0,0,0].item())
    print('')


### トレーニング

ここで、トレーニングデータローダーを準備しました。 以下では、トレーニング手順を簡単に示します。 完全なトレーニングには多くの時間がかかるため、最初のいくつかの反復を示します。

デバイスを設定します。 この場合、 "cpu".

In [ ]:
use_cuda = False
device = torch.device("cuda" if use_cuda else "cpu")

トレーニングする最大エポックを設定します。 デモンストレーションのために、max_epochs 1を設定しますが、数十個のエポックを使用することをお勧めします。

In [ ]:
max_epochs = 1

モデルをデバイスに転送

In [ ]:
model = model.to(device)

オプティマイザーを設定します。 ここでは、Adamを使用します。

In [ ]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9,0.999), weight_decay=1.0e-4)

損失関数を設定し、デバイスに転送します。

In [ ]:
criterion = Criterion().to(device)

トレーニングの繰り返し。 いくつかの反復のみを示します。

In [ ]:
for epoch in range(max_epochs):
    for idx, (image, label) in enumerate(loader):
        image, label = image.to(device), label.to(device)
        
        # At the begining of each iteraton, clear accumulated gradient for the network parameters.
        optimizer.zero_grad()
        
        # Input the image to the model and get the prediction.
        output = model(image)
        
        # Calculate the loss function comparing the prediction and the ground truth
        loss = criterion(output, label)
        
        # Backpropagate the error signal through the model to calculate gradients.
        loss.backward()
        
        # Update the model parameters using the calculated gradients.
        optimizer.step()

        print('Epoch #%d, batch #%d: loss=%f' % (epoch, idx, loss.item()))
        
        # For demonstration, stop iteration at iter5
        if idx == 4:
            break


### 学習したモデルを保存する

出力ディレクトリを設定する

In [ ]:
import os
output_dir = '../learned_weights/'

if not os.path.isdir(output_dir):
    os.mkdir(output_dir)

学習した重みをdictionaryとして取得します。

In [ ]:
dict_weights = model.state_dict()
for key, weight in dict_weights.items():
    print('%s\t%s' % (key, weight.size()))

dictionaryを保存します。

In [ ]:
torch.save(dict_weights, output_dir + '/demo_building_vggs.torch')

### 推論

設定。 切り取りウィンドウを16ストライドずつスライドさせて、テスト画像を切り取ります。

In [ ]:
patch_size = 64
aoi_size = 16
stride = 16

<img src="../fig/fig_test_patch_small.png" width="300">

テスト領域の画像を準備する

In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from PIL import Image
import numpy as np

use_cuda = False
device = torch.device("cuda" if use_cuda else "cpu")

# Load image
fpath_test_image = '../dataset/building_vmnih/test/image/22828930_15.tiff'
image = np.array(Image.open(fpath_test_image))
org_height, org_width, _ = image.shape

# Normalize
mean = np.mean(image, axis=(0,1), keepdims=True)
std = np.std(image, axis=(0,1), keepdims=True)
image = (image - mean) / std

# To tensor
image = image.transpose([2,0,1])    # Swap dimensions, (H,W,B) to (B,H,W)
image = image[np.newaxis,:,:,:]    # Add batch dimension (N,B,H,W)
image = torch.from_numpy(image).float()    # Convert to float tensor

ネットワークは入力パッチの中央領域の推定結果を出力するため、画像のエッジ領域を推定するために、入力パッチの一部が有効な画像領域外にあります。 したがって、パディングを使用して元の画像を拡大する必要があります。

<img src="../fig/fig_test_padding_small.png" width="600">

In [ ]:
# Reflection padding
pad_size = int((64-16)/2)
pad = nn.ReflectionPad2d(pad_size)
image = pad(image)
_, _, height, width = image.size()



モデルを設定し、トレーニング済みの重みを読み込む

In [ ]:
# Set model
model = VGGS().to(device)

# Load learned weights
learned_weights = torch.load('../learned_weights/demo_building_vggs.torch')
model.load_state_dict(learned_weights)

In [ ]:

# Set place holder
prob_map = np.zeros([height, width])
count_map = np.zeros([height, width])

# Crop patches in sliding manner, and apply the prediction model
num_tiles_x = (width - patch_size) // stride + 2
num_tiles_y = (height - patch_size) // stride + 2

for iy in range(num_tiles_y):
    for ix in range(num_tiles_x):
        print('(%d/%d, %d/%d)' % (iy, num_tiles_y, ix, num_tiles_x))
        ulx = ix * stride
        uly = iy * stride
        lrx = ulx + patch_size
        lry = uly + patch_size
        
        if lrx > width:
            ulx = width - patch_size
            lrx = width
            
        if lry > height:
            uly = height - patch_size
            lry = height
            
        patch = image[:, :, uly:lry, ulx:lrx]
        patch = patch.to(device)
        
        logit = model(patch)
        prob = F.sigmoid(logit)
        
        stx = ulx + int((patch_size - aoi_size) / 2)
        sty = uly + int((patch_size - aoi_size) / 2)
        prob_map[sty:sty+aoi_size, stx:stx+aoi_size] += prob.detach().cpu().numpy().squeeze()
        count_map[sty:sty+aoi_size, stx:stx+aoi_size] += np.ones([aoi_size,aoi_size])

# Eliminate padded region
prob_map = prob_map[pad_size:pad_size+org_height, pad_size:pad_size+org_width]
count_map = count_map[pad_size:pad_size+org_height, pad_size:pad_size+org_width]

# Take average for overlaped regions
result = prob_map / count_map


推定結果を確認する

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

fpath_test_label = '../dataset/building_vmnih/test/label/22828930_15.tif'
label = np.array(Image.open(fpath_test_label))[:,:,0]

fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(121)
ax.imshow(result, interpolation='none')
ax.set_xticks([])
ax.set_yticks([])
fig.show()
ax.set_title('Estimation')

ax = fig.add_subplot(122)
ax.imshow(label, interpolation='none')
ax.set_xticks([])
ax.set_yticks([])
ax.set_title('Ground truth')

fig.show()


In [ ]:
import numpy as np
pred = (result > 0.5)
label = (label > 0.5)

TP = np.sum(pred*label)
T = np.sum(label)
P = np.sum(pred)
IoU = float(TP) / (T+P-TP) * 100
print('IoU: %.2f%%' % IoU)

次に、Datasetクラスにデータ拡張（ランダムな回転と反転）を追加します。

In [ ]:
import numpy as np
from torch.utils.data import Dataset
import random
from PIL import Image

class Buildings(Dataset):
    def __init__(self, fpath_image_npy, fpath_label_npy, augmentation=False):
        self.images = np.load(fpath_image_npy)
        self.labels = np.load(fpath_label_npy)
        self.augmentation = augmentation
        
    def __getitem__(self, index):
        image = self.images[index]
        label = self.labels[index]
        
        # Augmentation
        if self.augmentation:
            # Randomly choose rotation angle and flipping direction
            rotation = random.choice([0, 90, 180, 270])
            flip = random.choice(['H', 'V', 'N'])
            
            # Apply transformation for image
            image = self._rotate(image, rotation)
            image = np.array(self._flip(image, flip))

            # Also, apply the same transformation for label
            label = self._rotate(label, rotation)
            label = np.array(self._flip(label, flip))
        
        image = image.transpose([2,0,1])    # (H,W,B) to (B,H,W)
        
        image_tensor = torch.from_numpy(image).float()
        label_tensor = torch.from_numpy(label).long()
        
        return image_tensor, label_tensor
    
    def __len__(self):
        return len(self.images)
    
    # Rotation function
    def _rotate(self, image, rotation):
        if rotation == 0:
            return image
        elif rotation == 90:
            image = image.swapaxes(0,1)
            return np.flip(image, axis=0)
        elif rotation == 180:
            image = np.flip(image, axis=0)
            return np.flip(image, axis=1)
        elif rotation == 270:
            image = image.swapaxes(0,1)
            return np.flip(image, axis=1)
        else:
            raise RuntimeError

    # Flip function
    def _flip(self, image, direction):
        if direction == 'H':
            return np.flip(image, axis=1)
        elif direction == 'V':
            return np.flip(image, axis=0)
        elif direction == 'N':
            return image


データセットからいくつかのパッチをサンプリングして、拡張性を確認します。

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from matplotlib.gridspec import GridSpec
grid = GridSpec(nrows=2, ncols=10)


mean = np.array([75, 75, 75])
std = np.array([50, 50, 50])

mean = mean[:, np.newaxis, np.newaxis]
std = std[:,np.newaxis, np.newaxis]

dataset = Buildings('../dataset/building_vmnih/train/patches/sat.npy', '../dataset/building_vmnih/train/patches/map.npy', augmentation=True)

fig = plt.figure(figsize=(20,4))
for i in range(10):
    image_tensor, label_tensor = dataset[3]
    image_patch = image_tensor.detach().numpy() * std + mean
    label_patch = label_tensor.detach().numpy()
    
    image_patch = image_patch.transpose([1,2,0])
    image_patch = image_patch.clip(0,255).astype(np.uint8)

    ax = fig.add_subplot(grid[0,i])
    ax.imshow(image_patch, interpolation='none')
    ax.set_xticks([])
    ax.set_yticks([])

    ax = fig.add_subplot(grid[1,i])
    ax.imshow(label_patch, interpolation='none')
    ax.set_xticks([])
    ax.set_yticks([])
fig.show()

**より多くのエポックを持つ大規模データを使用して、訓練されたモデルの結果を表示する**

<img src= "../fig/bulding_detct.png" width =600 >

このような手法は、ラベル付きデータの量が限られている場合に特に効果的です。
他の方法があります 
- 転移学習
- データ融合
- 拡張畳み込み
